### Configuration et préparation des données

In [ ]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [ ]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

# Partie 7: GEPA en pratique

## 7.1 Introduction à GEPA

**GEPA** (Genetic-Pareto Algorithm) est l'optimiseur le plus sophistiqué de DSPy. Il combine plusieurs techniques avancées pour améliorer automatiquement vos prompts.

### Qu'est-ce que GEPA?

GEPA utilise une approche inspirée de l'évolution biologique :

1. **🧬 Algorithmes génétiques** : Génère des "populations" de prompts qui évoluent
2. **🤔 Réflexion LLM** : Utilise un LLM pour analyser les erreurs et proposer des améliorations
3. **📊 Optimisation Pareto** : Équilibre plusieurs objectifs (précision, concision, etc.)
4. **🔄 Itérations adaptatives** : Apprend de ses erreurs pour s'améliorer

### Comment ça fonctionne?

```
┌─────────────────────────────────────────────────────────┐
│ 1. POPULATION INITIALE                                  │
│    Génère plusieurs variantes de prompts               │
│    ↓                                                    │
│ 2. ÉVALUATION                                          │
│    Teste chaque variante sur les données d'entraînement│
│    ↓                                                    │
│ 3. SÉLECTION                                           │
│    Garde les meilleurs (front de Pareto)               │
│    ↓                                                    │
│ 4. RÉFLEXION                                           │
│    LLM analyse les erreurs et propose des améliorations│
│    ↓                                                    │
│ 5. MUTATION                                            │
│    Génère de nouvelles variantes basées sur la réflexion│
│    ↓                                                    │
│ 6. RÉPÈTE jusqu'à convergence                          │
└─────────────────────────────────────────────────────────┘
```

### Pourquoi GEPA est différent?

| Optimiseur | Approche | Réflexion LLM | Amélioration typique |
|------------|----------|---------------|---------------------|
| BootstrapFewShot | Exemples fixes | ❌ Non | 5-15% |
| MIPRO | Variations systématiques | ❌ Non | 10-25% |
| **GEPA** | **Évolution + Réflexion** | **✅ Oui** | **15-30%** |

### Nouveautés dans DSPy 3.0+

GEPA a été intégré directement dans DSPy et nécessite maintenant :
- **reflection_lm** : Un modèle LLM dédié à l'analyse des erreurs
- **auto** : Niveau d'optimisation ('light', 'medium', 'heavy')
- **Métrique compatible** : Doit accepter les paramètres GEPA

## 7.2 Configuration de GEPA

### Prérequis

Avant d'utiliser GEPA, vous devez avoir :

1. ✅ **Données d'entraînement** : Au moins 15-20 exemples de qualité
2. ✅ **Données de validation** : 5-10 exemples séparés pour éviter le surapprentissage
3. ✅ **Métrique d'évaluation** : Fonction retournant un score entre 0 et 1
4. ✅ **Module à optimiser** : Votre module DSPy de base
5. ✅ **Temps** : 10-30 minutes selon le niveau d'optimisation

### Configuration du modèle de réflexion

Le **reflection_lm** est un LLM utilisé par GEPA pour analyser les erreurs et proposer des améliorations. Il est distinct du modèle principal.

**Recommandations** :
- Utiliser un modèle avec bonne capacité de raisonnement
- Température élevée (1.0) pour plus de créativité
- Max tokens élevé (8000+) pour des analyses détaillées

In [ ]:
print("🔧 Configuration de GEPA\n")

# 1. Configurer le modèle principal (celui qu'on optimise)
lm_main = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

dspy.configure(lm=lm_main)

# 2. Configurer le modèle de réflexion pour GEPA
# Important: Température élevée pour plus de créativité dans l'analyse
reflection_lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=1.0,      # Haute température = plus de créativité
    max_tokens=8000       # Tokens élevés = analyses détaillées
)

print("✅ Modèle principal configuré: llama3.1:8b (temp=0.3)")
print("✅ Modèle de réflexion configuré: llama3.1:8b (temp=1.0)")
print()

# 3. Préparer les données au format DSPy
train_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in trainset
]

val_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in valset
]

print(f"✅ Données préparées:")
print(f"   - Entraînement: {len(train_examples)} exemples")
print(f"   - Validation: {len(val_examples)} exemples")

## 7.3 Optimisation avec GEPA (mode 'light')

Le mode **'light'** est le niveau d'optimisation le plus rapide de GEPA. Il est idéal pour :
- Première expérimentation avec GEPA
- Tests rapides (5-10 minutes)
- Validation du concept
- Ressources limitées

### Niveaux d'optimisation GEPA

| Niveau | Temps estimé | Appels LLM | Amélioration | Usage |
|--------|-------------|------------|--------------|-------|
| **light** | 5-10 min | ~200-400 | 10-20% | Tests, prototypage |
| **medium** | 10-20 min | ~400-800 | 15-25% | Production légère |
| **heavy** | 20-40 min | ~800-1600 | 20-30% | Maximum performance |

### Exemple pratique

In [ ]:
from dspy.teleprompt import GEPA

print("=" * 70)
print("🧬 OPTIMISATION GEPA - MODE 'LIGHT'")
print("=" * 70)
print()

# 1. Créer le module à optimiser
print("1️⃣ Création du module de base...")
baseline_classifier = SimpleTicketClassifier()

# 2. Évaluer AVANT optimisation
print("2️⃣ Évaluation AVANT optimisation...")
score_before = evaluate_module(baseline_classifier, val_examples, exact_match_metric)
print(f"   📊 Score baseline: {score_before:.2%}\n")

# 3. Configurer l'optimiseur GEPA
print("3️⃣ Configuration de l'optimiseur GEPA...")
optimizer = GEPA(
    metric=exact_match_metric,
    auto='light',                    # Mode rapide
    reflection_lm=reflection_lm      # Modèle pour l'analyse des erreurs
)
print("   ✅ Optimiseur configuré (mode: light)\n")

# 4. Lancer l'optimisation
print("4️⃣ Lancement de l'optimisation GEPA...")
print("   ⏰ Cela va prendre 5-10 minutes avec Ollama")
print("   ☕ C'est le moment de prendre un café!\n")

try:
    optimized_classifier = optimizer.compile(
        student=baseline_classifier,
        trainset=train_examples,
        valset=val_examples
    )
    
    print("\n✅ Optimisation GEPA terminée!\n")
    
    # 5. Évaluer APRÈS optimisation
    print("5️⃣ Évaluation APRÈS optimisation...")
    score_after = evaluate_module(optimized_classifier, val_examples, exact_match_metric)
    print(f"   📊 Score optimisé: {score_after:.2%}\n")
    
    # 6. Calculer l'amélioration
    improvement = ((score_after - score_before) / score_before) * 100 if score_before > 0 else 0
    improvement_abs = score_after - score_before
    
    print("=" * 70)
    print("📈 RÉSULTATS DE L'OPTIMISATION")
    print("=" * 70)
    print(f"Score AVANT:      {score_before:.2%}")
    print(f"Score APRÈS:      {score_after:.2%}")
    print(f"Amélioration:     {improvement_abs:+.2%} ({improvement:+.1f}%)")
    print("=" * 70)
    
except Exception as e:
    print(f"\n❌ Erreur lors de l'optimisation GEPA: {e}")
    print("   Vérifiez que:")
    print("   - Ollama est en cours d'exécution")
    print("   - Le modèle llama3.1:8b est téléchargé")
    print("   - Vous avez suffisamment de mémoire disponible")

## 7.4 Analyser les prompts optimisés

Une des forces de GEPA est qu'il **génère des prompts explicites** que vous pouvez inspecter et comprendre. Cela permet de :
- 🔍 Voir ce que GEPA a changé
- 📚 Apprendre comment améliorer vos prompts manuellement
- ✅ Valider que les modifications ont du sens
- 🎓 Comprendre pourquoi la performance s'est améliorée

### Inspection des prompts

In [ ]:
print("🔍 Inspection des prompts optimisés par GEPA\n")

# Essayer d'accéder aux prompts optimisés
if hasattr(optimized_classifier, 'classifier'):
    predictor = optimized_classifier.classifier
    
    # 1. Signature optimisée
    if hasattr(predictor, 'extended_signature'):
        print("=" * 70)
        print("📝 SIGNATURE OPTIMISÉE")
        print("=" * 70)
        sig = predictor.extended_signature
        print(f"Docstring: {sig.__doc__}")
        print()
        
        # Afficher les champs
        if hasattr(sig, 'input_fields'):
            print("Champs d'entrée:")
            for name, field in sig.input_fields.items():
                desc = getattr(field, 'desc', 'N/A')
                print(f"  - {name}: {desc}")
        
        if hasattr(sig, 'output_fields'):
            print("\nChamps de sortie:")
            for name, field in sig.output_fields.items():
                desc = getattr(field, 'desc', 'N/A')
                print(f"  - {name}: {desc}")
        print()
    
    # 2. Exemples de démonstration
    if hasattr(predictor, 'demos') and predictor.demos:
        print("=" * 70)
        print("📚 EXEMPLES DE DÉMONSTRATION GÉNÉRÉS")
        print("=" * 70)
        print(f"Nombre d'exemples: {len(predictor.demos)}\n")
        
        # Afficher les 3 premiers exemples
        for i, demo in enumerate(predictor.demos[:3], 1):
            print(f"Exemple {i}:")
            print(f"  Ticket: {demo.ticket[:80]}...")
            if hasattr(demo, 'category'):
                print(f"  Catégorie: {demo.category}")
            if hasattr(demo, 'priority'):
                print(f"  Priorité: {demo.priority}")
            print()
    
    else:
        print("ℹ️ Aucun exemple de démonstration trouvé")
        print("   (GEPA peut optimiser uniquement les instructions)")
    
else:
    print("ℹ️ Structure du module différente")
    print("   L'optimisation s'est concentrée sur d'autres aspects")

print("=" * 70)
print("💡 Ce que GEPA a fait:")
print("=" * 70)
print("✅ Analysé les erreurs sur les données d'entraînement")
print("✅ Généré des variantes de prompts")
print("✅ Utilisé la réflexion LLM pour proposer des améliorations")
print("✅ Sélectionné la meilleure configuration via Pareto")
print("=" * 70)

## 7.5 Tester le classifier optimisé

Maintenant que GEPA a optimisé notre classifier, testons-le sur de nouveaux exemples pour voir la différence.

### Comparaison côte à côte

In [ ]:
print("🧪 Test comparatif: Baseline vs GEPA optimisé\n")

# Exemples de test
test_cases = [
    {
        'ticket': "Mon ordinateur portable ne s'allume plus, j'ai une réunion importante dans 1h",
        'expected_category': 'Hardware',
        'expected_priority': 'Urgent'
    },
    {
        'ticket': "Je voudrais accès à la base de données pour faire des analyses",
        'expected_category': 'Account',
        'expected_priority': 'Medium'
    },
    {
        'ticket': "Le WiFi est complètement HS dans tout le bâtiment!",
        'expected_category': 'Network',
        'expected_priority': 'Critical'
    },
    {
        'ticket': "Mon logiciel de comptabilité plante quand j'exporte en PDF",
        'expected_category': 'Software',
        'expected_priority': 'High'
    }
]

print("=" * 100)
print(f"{'Ticket':<50} | {'Attendu':<20} | {'Baseline':<20} | {'GEPA':<20}")
print("=" * 100)

correct_baseline = 0
correct_gepa = 0

for test in test_cases:
    ticket = test['ticket']
    expected = f"{test['expected_category']}/{test['expected_priority']}"
    
    # Prédiction baseline
    pred_baseline = baseline_classifier(ticket=ticket)
    baseline_result = f"{pred_baseline.category}/{pred_baseline.priority}"
    baseline_match = (pred_baseline.category == test['expected_category'] and 
                     pred_baseline.priority == test['expected_priority'])
    
    # Prédiction GEPA
    pred_gepa = optimized_classifier(ticket=ticket)
    gepa_result = f"{pred_gepa.category}/{pred_gepa.priority}"
    gepa_match = (pred_gepa.category == test['expected_category'] and 
                 pred_gepa.priority == test['expected_priority'])
    
    if baseline_match:
        correct_baseline += 1
    if gepa_match:
        correct_gepa += 1
    
    # Afficher avec indicateurs de succès
    baseline_icon = "✅" if baseline_match else "❌"
    gepa_icon = "✅" if gepa_match else "❌"
    
    print(f"{ticket[:48]:<50} | {expected:<20} | {baseline_icon} {baseline_result:<17} | {gepa_icon} {gepa_result:<17}")

print("=" * 100)
print(f"Précision sur les exemples de test:")
print(f"  Baseline: {correct_baseline}/{len(test_cases)} ({correct_baseline/len(test_cases)*100:.0f}%)")
print(f"  GEPA:     {correct_gepa}/{len(test_cases)} ({correct_gepa/len(test_cases)*100:.0f}%)")
print("=" * 100)

## 7.6 Conseils pour optimiser avec GEPA

### 7.6.1 Qualité des données

La qualité de l'optimisation GEPA dépend **fortement** de vos données :

✅ **Bonnes pratiques** :
- Minimum 15-20 exemples d'entraînement (idéalement 30-50)
- Exemples **diversifiés** couvrant tous les cas d'usage
- Labels **corrects** et **cohérents**
- Données de validation **séparées** du trainset

❌ **À éviter** :
- Trop peu d'exemples (<10)
- Exemples répétitifs ou très similaires
- Labels incohérents ou ambigus
- Utiliser les mêmes données pour train et validation

### 7.6.2 Choix de la métrique

Votre **métrique détermine ce que GEPA optimise** :

```python
# Métrique stricte (tout ou rien)
def exact_match(example, prediction, trace=None, pred_name=None, pred_trace=None):
    return 1.0 if (prediction.category == example.category and 
                   prediction.priority == example.priority) else 0.0

# Métrique avec crédit partiel (souvent meilleure pour GEPA)
def partial_match(example, prediction, trace=None, pred_name=None, pred_trace=None):
    category_match = prediction.category == example.category
    priority_match = prediction.priority == example.priority
    
    if category_match and priority_match:
        return 1.0
    elif category_match:
        return 0.7  # Catégorie plus importante
    elif priority_match:
        return 0.5
    else:
        return 0.0
```

💡 **Conseil** : Les métriques avec crédit partiel donnent généralement de meilleurs résultats car elles fournissent plus de signal d'apprentissage à GEPA.

### 7.6.3 Choix du niveau d'optimisation

| Situation | Niveau recommandé | Raison |
|-----------|------------------|---------|
| Première expérimentation | **light** | Test rapide du concept |
| Prototype pour démo | **light** | Balance vitesse/performance |
| Application production (non-critique) | **medium** | Bon compromis |
| Application production (critique) | **heavy** | Maximum de performance |
| Recherche / benchmark | **heavy** | Explorer les limites |

### 7.6.4 Configuration du modèle de réflexion

Le **reflection_lm** influence la qualité des améliorations :

✅ **Bonnes pratiques** :
- Utiliser un modèle avec bonne capacité de raisonnement
- Température élevée (0.8-1.2) pour la créativité
- Max tokens élevé (6000-10000) pour analyses détaillées
- Peut être le même modèle que le modèle principal

❌ **À éviter** :
- Modèles trop petits (<7B paramètres)
- Température trop basse (<0.5)
- Max tokens trop faible (<4000)

### 7.6.5 Éviter le surapprentissage

GEPA peut **surapprendre** sur les données d'entraînement :

✅ **Prévention** :
- Toujours avoir un **valset séparé**
- Valider sur de **nouvelles données** après optimisation
- Comparer les scores train vs validation
- Si grand écart : vos données ne sont pas assez diversifiées

```python
# Bon: Évaluation sur données séparées
score_train = evaluate_module(optimized, train_examples, metric)
score_val = evaluate_module(optimized, val_examples, metric)

if score_train - score_val > 0.2:
    print("⚠️ Possible surapprentissage!")
```

## 7.7 Troubleshooting et erreurs courantes

### Erreur 1: `TypeError: GEPA.__init__() got an unexpected keyword argument`

**Symptôme** :
```
TypeError: GEPA.__init__() got an unexpected keyword argument 'breadth'
```

**Cause** : Utilisation de paramètres de l'ancienne API GEPA (pre-3.0)

**Solution** :
```python
# ❌ Ancienne API (ne fonctionne plus)
optimizer = GEPA(
    metric=my_metric,
    breadth=10,
    depth=3
)

# ✅ Nouvelle API (DSPy 3.0+)
optimizer = GEPA(
    metric=my_metric,
    auto='light',  # ou 'medium', 'heavy'
    reflection_lm=reflection_lm
)
```

### Erreur 2: `TypeError: metric() missing required positional argument`

**Symptôme** :
```
TypeError: metric() missing 2 required positional arguments: 'pred_name' and 'pred_trace'
```

**Cause** : Métrique pas compatible avec l'API GEPA

**Solution** : Ajouter les paramètres optionnels à votre métrique
```python
# ❌ Ancienne signature
def my_metric(example, prediction):
    return 1.0 if prediction.category == example.category else 0.0

# ✅ Nouvelle signature (compatible GEPA)
def my_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    return 1.0 if prediction.category == example.category else 0.0
```

### Erreur 3: Ollama timeout ou erreur de connexion

**Symptôme** :
```
ConnectionError: Failed to connect to Ollama
```

**Causes possibles** :
1. Ollama n'est pas démarré
2. Modèle non téléchargé
3. Mémoire insuffisante

**Solutions** :
```bash
# 1. Vérifier qu'Ollama tourne
ollama list

# 2. Démarrer Ollama si nécessaire
ollama serve

# 3. Télécharger le modèle
ollama pull llama3.1:8b

# 4. Vérifier la mémoire disponible
# GEPA + Ollama nécessite ~8-12 GB RAM
```

### Erreur 4: GEPA ne s'améliore pas

**Symptôme** : Score après optimisation ≈ score avant

**Causes possibles** :
1. Données d'entraînement insuffisantes ou de mauvaise qualité
2. Métrique mal définie
3. Tâche trop difficile pour le modèle
4. Module déjà bien optimisé

**Solutions** :
```python
# Vérifier la qualité des données
print(f"Nombre d'exemples train: {len(trainset)}")
print(f"Nombre d'exemples val: {len(valset)}")

# Vérifier la métrique
for ex in trainset[:5]:
    pred = baseline(ticket=ex['ticket'])
    score = metric(ex, pred)
    print(f"Score: {score} | Pred: {pred.category}/{pred.priority} | Truth: {ex['category']}/{ex['priority']}")

# Essayer un niveau plus élevé
optimizer = GEPA(
    metric=metric,
    auto='heavy',  # au lieu de 'light'
    reflection_lm=reflection_lm
)
```

### Erreur 5: Out of Memory (OOM)

**Symptôme** : Ollama ou Python crash avec erreur de mémoire

**Solutions** :
1. Utiliser un modèle plus petit : `mistral:7b` au lieu de `llama3.1:8b`
2. Réduire `max_tokens` du reflection_lm
3. Utiliser `auto='light'` au lieu de `medium` ou `heavy`
4. Fermer les autres applications
5. Utiliser une API cloud au lieu d'Ollama local

## 7.8 Résumé de la Partie 7

### Ce que nous avons appris

1. **GEPA : L'optimiseur le plus sophistiqué**
   - Combine algorithmes génétiques + réflexion LLM + Pareto
   - Amélioration typique : 15-30%
   - Nécessite un modèle de réflexion (reflection_lm)

2. **Configuration essentielle**
   - `auto` : 'light', 'medium', ou 'heavy'
   - `reflection_lm` : Modèle pour analyser les erreurs (temp=1.0, max_tokens=8000)
   - `metric` : Doit accepter les paramètres GEPA (trace, pred_name, pred_trace)

3. **Niveaux d'optimisation**
   - **light** : 5-10 min, ~200-400 appels LLM, 10-20% amélioration
   - **medium** : 10-20 min, ~400-800 appels LLM, 15-25% amélioration
   - **heavy** : 20-40 min, ~800-1600 appels LLM, 20-30% amélioration

4. **Inspection des résultats**
   - Voir les prompts optimisés
   - Comprendre les changements
   - Valider les améliorations

5. **Bonnes pratiques**
   - Données : 15-20+ exemples diversifiés
   - Métrique : Crédit partiel souvent meilleur
   - Validation : Toujours sur données séparées
   - Modèle : ≥7B paramètres recommandé

6. **Troubleshooting**
   - Erreurs d'API : Vérifier la signature de la métrique
   - Pas d'amélioration : Vérifier qualité des données
   - OOM : Réduire niveau ou utiliser modèle plus petit

### Points clés à retenir

- ✅ **GEPA est puissant** mais nécessite temps et ressources
- ✅ **Commencer avec 'light'** pour tester le concept
- ✅ **Qualité des données = qualité des résultats**
- ✅ **Toujours valider** sur données séparées
- ✅ **Inspecter les prompts** pour comprendre les améliorations

### Quand utiliser GEPA?

| Situation | GEPA? | Alternative |
|-----------|-------|-------------|
| Prototypage rapide | ❌ Non | Module simple |
| Tests initiaux | ❌ Non | BootstrapFewShot |
| Application production (non-critique) | ⚠️ Peut-être | MIPRO |
| Application production (critique) | ✅ Oui | GEPA medium/heavy |
| Recherche / Maximum performance | ✅ Oui | GEPA heavy |
| Ressources limitées | ❌ Non | BootstrapFewShot |

### Workflow recommandé

```python
# Phase 1: Baseline
classifier = SimpleTicketClassifier()
score_baseline = evaluate(classifier, valset, metric)

# Phase 2: Optimisation simple
from dspy.teleprompt import BootstrapFewShot
optimizer_simple = BootstrapFewShot(metric=metric)
classifier_simple = optimizer_simple.compile(classifier, trainset)
score_simple = evaluate(classifier_simple, valset, metric)

# Phase 3: GEPA (si amélioration justifie le temps)
if score_simple < target_score:
    from dspy.teleprompt import GEPA
    optimizer_gepa = GEPA(metric=metric, auto='light', reflection_lm=reflection_lm)
    classifier_gepa = optimizer_gepa.compile(classifier, trainset, valset)
    score_gepa = evaluate(classifier_gepa, valset, metric)
    
    # Phase 4: GEPA heavy si nécessaire
    if score_gepa < target_score:
        optimizer_heavy = GEPA(metric=metric, auto='heavy', reflection_lm=reflection_lm)
        classifier_final = optimizer_heavy.compile(classifier, trainset, valset)
```

### Ressources additionnelles

- 📄 [Paper GEPA (arXiv)](https://arxiv.org/abs/2507.19457)
- 💻 [GitHub GEPA](https://github.com/gepa-ai/gepa)
- 📖 [Documentation DSPy](https://dspy-docs.vercel.app/)

### Prochaines étapes

- **Partie 8** : Conclusion et mise en production